In [ ]:
import logging
import time
from collections import defaultdict

import sqlalchemy
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import (
    Column,
    Float,
    Integer,
    String,
    create_engine,
    func,
    select,
)
from chembl_webresource_client.new_client import new_client

In [2]:
# Set logging level to INFO
logging.getLogger().setLevel(logging.INFO)

Base = declarative_base()

engine = create_engine("sqlite:///compounds.db", echo=False)
Session = sessionmaker(bind=engine)

In [3]:
def get_chembl_molecules(
    n_compounds: int = 2,
    start_id: int = 1,
):
    chembl_ids = [f"CHEMBL{id}" for id in range(start_id, start_id + n_compounds)]

    molecule = new_client.molecule
    activity = new_client.activity
    target = new_client.target

    # Suppress INFO logs from the chembl package while keeping global INFO level
    logging.getLogger("chembl_webresource_client").setLevel(logging.WARNING)

    # --------------------
    # 1) Fetch molecules
    # --------------------
    mols = list(
        molecule.filter(molecule_chembl_id__in=chembl_ids).only(
            [
                "molecule_chembl_id",
                "molecule_structures",
                "pref_name",
                "molecule_properties",
            ]
        )
    )

    # From the molecules in mols, extract the digits after "CHEMBL" to see which IDs were found
    chembl_ids_found = set()
    for mol in mols:
        chembl_id = mol.get("molecule_chembl_id", "")
        if chembl_id.startswith("CHEMBL"):
            chembl_ids_found.add(int(chembl_id.replace("CHEMBL", "")))
    chembl_ids_found_str = ", ".join(map(str, sorted(chembl_ids_found)))

    logging.info(
        f"Of the {n_compounds} ChEMBL IDs "
        f"({start_id}-{start_id + n_compounds - 1}), {len(mols)} are compounds: "
        f"ChEMBL IDs {chembl_ids_found_str}"
    )

    if not mols:
        return []

    mol_ids_present = [m["molecule_chembl_id"] for m in mols]

    # ---------------------------------
    # 2) Bulk fetch activities → targets
    # ---------------------------------
    acts = activity.filter(
        molecule_chembl_id__in=mol_ids_present,
        target_organism="Homo sapiens",
        standard_type="IC50",
        assay_type="B",
        document_year__lt=2010,
    ).only(
        [
            "molecule_chembl_id",
            "target_chembl_id",
        ]
    )
    logging.info(f"Activities: {len(acts)}")

    mol_to_target_ids = defaultdict(set)

    for a in acts:
        try:
            mol_to_target_ids[a["molecule_chembl_id"]].add(a["target_chembl_id"])
        except Exception as e:
            print(e)

    # ---------------------------------
    # 3) Fetch target metadata (bulk)
    # ---------------------------------
    all_target_ids = sorted(
        {tid for tids in mol_to_target_ids.values() for tid in tids}
    )

    targets = {}
    if all_target_ids:
        for t in target.filter(target_chembl_id__in=all_target_ids).only(
            [
                "target_chembl_id",
                "pref_name",
                "target_type",
                "organism",
            ]
        ):
            targets[t["target_chembl_id"]] = t
    logging.info(f"Fetched metadata for {len(targets)} targets.")

    # ---------------------------------
    # 4) Attach targets to molecules
    # ---------------------------------

    for m in mols:
        t_ids = mol_to_target_ids.get(m["molecule_chembl_id"], [])
        m["targets"] = [targets[tid] for tid in t_ids if tid in targets]

    return mols


In [4]:
def save_compounds_to_db(molecules: list[dict]) -> tuple[int, int, int]:
    """Save multiple compounds and their targets to the database efficiently avoiding duplicate Targets."""
    # collect all target ids present in incoming molecules
    all_target_ids = {
        t["target_chembl_id"]
        for m in molecules
        for t in m.get("targets", [])
        if t.get("target_chembl_id")
    }

    n_mols_saved = 0
    n_targets_saved = 0
    n_compounds_targets_saved = 0

    with Session() as db_session:
        try:
            # preload existing targets into a mapping chembl_id -> Target instance
            existing_targets = {}
            if all_target_ids:
                rows = (
                    db_session.query(Target)
                    .filter(Target.target_chembl_id.in_(list(all_target_ids)))
                    .all()
                )
                existing_targets = {r.target_chembl_id: r for r in rows}

            for mol in molecules:
                chembl_id = mol.get("molecule_chembl_id")
                pref_name = mol.get("pref_name")
                props = mol.get("molecule_properties", {}) or {}
                compound = Compound(
                    chembl_id=chembl_id,
                    sml=mol.get("molecule_structures", {}).get("canonical_smiles"),
                    pref_name=pref_name,
                    molwt=props.get("full_molweight"),
                    tpsa=props.get("tpsa"),
                    num_h_acceptors=props.get("num_h_acceptors"),
                    num_h_donors=props.get("num_h_donors"),
                    num_ro5=props.get("num_ro5_violations"),
                    mol_logp=props.get("alogp"),
                )

                with db_session.begin_nested():
                    db_session.add(compound)
                    db_session.flush()  # get compound.id

                    for target_data in mol.get("targets", []):
                        target_id = target_data.get("target_chembl_id")
                        if not target_id:
                            continue

                        # reuse existing Target if present
                        target_obj = existing_targets.get(target_id)
                        if target_obj is None:
                            # create new Target, add and flush to get id, then cache it
                            target_obj = Target(
                                organism=target_data.get("organism"),
                                pref_name=target_data.get("pref_name"),
                                target_chembl_id=target_id,
                                target_type=target_data.get("target_type"),
                            )
                            db_session.add(target_obj)
                            db_session.flush()  # populates target_obj.id
                            existing_targets[target_id] = target_obj
                            n_targets_saved += 1

                        # create association
                        compound_target = CompoundTarget(
                            compound_id=compound.id,
                            target_id=target_obj.id,
                        )
                        db_session.add(compound_target)
                        n_compounds_targets_saved += 1

                n_mols_saved += 1

            # outer commit happens when exiting the with Session() context
        except IntegrityError as e:
            # handle unexpected integrity issues gracefully
            logging.info(f"IntegrityError while saving: {e}")
            db_session.rollback()
        except SQLAlchemyError:
            logging.exception("Database error saving compounds")
            db_session.rollback()
            raise

    return n_mols_saved, n_targets_saved, n_compounds_targets_saved


In [5]:
def init_db():
    """Create database tables (call once at app startup or from scripts/tests)."""
    Base.metadata.create_all(engine)

In [6]:
def reset_db():
    """Drop all database tables (use with caution)."""
    Base.metadata.drop_all(engine)

In [7]:
class Compound(Base):
    __tablename__ = "compound"

    id = Column(Integer, primary_key=True)
    chembl_id = Column(Integer, unique=True)
    sml = Column(String)
    pref_name = Column(String)
    molwt = Column(Float)  # MolWt
    tpsa = Column(Float)  # TPSA
    num_h_acceptors = Column(Integer)  # NumHAcceptors
    num_h_donors = Column(Integer)  # NumHDonors
    num_ro5 = Column(Integer)  # NumRo5
    mol_logp = Column(Float)  # MolLogP


In [8]:
class Target(Base):
    __tablename__ = "target"

    id = Column(Integer, primary_key=True)
    organism = Column(String)
    pref_name = Column(String)
    target_chembl_id = Column(String, unique=True)
    target_type = Column(String)


In [9]:
# Join table
class CompoundTarget(Base):
    __tablename__ = "compound_target"

    id = Column(Integer, primary_key=True)
    compound_id = Column(Integer, sqlalchemy.ForeignKey("compound.id"))
    target_id = Column(Integer, sqlalchemy.ForeignKey("target.id"))

In [10]:
def run_queries():
    """Run the required queries against the Pokemon database and print the results."""
    with Session() as db_session:
        # 1. Find the count of all distinct counts of compound targets (ex: Voltage-gated inwardly rectifying potassium channel KCNH2:14, Neuronal acetylcholine receptor subunit alpha-3/Neuronal acetylcholine receptor subunit alpha-7: 5).
        # For each compound, concatenate its targets with a slash.
        # Then, count how many distinct such tuples exist in the database.

        # Build per-compound target combo subquery:
        # as correlated scalar subquery: group_concat over an ordered selection of types to ensure consistent ordering.

        # correlated inner select returning pref_name for the current Compound, ordered
        inner = (
            select(Target.pref_name)
            .select_from(Target.__table__.join(CompoundTarget.__table__, CompoundTarget.target_id == Target.id))
            .where(CompoundTarget.compound_id == Compound.id)
            .order_by(func.lower(Target.pref_name))
            .correlate(Compound)
        )

        # name the derived table so the outer group_concat can select from it
        ordered_targets = inner.subquery("ordered_targets")

        # aggregate the ordered names with a '\' separator
        target_combo_subq = select(func.group_concat(ordered_targets.c.pref_name, "\\")).scalar_subquery()

        # Create a subquery that selects each compound's id and its target combination.
        target_combinations = db_session.query(
            Compound.id.label("compound_id"),
            target_combo_subq.label("target_combo"),
        ).subquery()

        # Create a list of distinct type combinations and their counts
        # where each is a tuple like (target_combo, num_compounds)
        compound_targets = (
            db_session.query(
            target_combinations.c.target_combo,
            func.count().label("num_compounds"),
            func.group_concat(Compound.chembl_id, ", ").label("chembl_ids"),
            )
            .join(Compound, Compound.id == target_combinations.c.compound_id)
            .group_by(target_combinations.c.target_combo)
            .order_by(target_combinations.c.target_combo)
            .all()
        )
        
        n_compound_by_target = 0
        logging.info("1. Distinct compound target combinations and their counts:")
        for target_combo, count, chembl_ids in compound_targets:
            logging.info(f"    {target_combo}: {count} (Compounds: {chembl_ids})")
            n_compound_by_target += count
        logging.info(
            f"    Total compounds counted by target combinations: {n_compound_by_target}"
        )


In [ ]:
# Reset database (uncomment to start fresh)
reset_db()

# Ensure tables exist
init_db()

# Measure how long it takes to fetch ChEMBL molecules
start = time.time()
mols = get_chembl_molecules(
    n_compounds=10,
    start_id=1,  # Has targets
    # start_id=3430873, # Not a molecule
)

end = time.time()
logging.info(f"Fetched {len(mols)} molecules in {end - start:.2f} seconds.")

start = time.time()

n_mols_saved, n_targets_saved, n_compounds_targets_saved = save_compounds_to_db(
    mols
)
logging.info(
    f"Saved {n_mols_saved} compounds, "
    f"{n_targets_saved} targets, "
    f"{n_compounds_targets_saved} compound-target associations to the database, "
    f"in {time.time() - start:.2f} seconds."
)

run_queries()

pass


INFO:root:Of the 10 ChEMBL IDs (1-10), 9 are compounds: ChEMBL IDs 1, 2, 3, 4, 5, 6, 8, 9, 10
INFO:root:Activities: 187
INFO:root:Fetched metadata for 37 targets.
INFO:root:Fetched 9 molecules in 0.02 seconds.
INFO:root:Saved 9 compounds, 37 targets, 43 compound-target associations to the database, in 0.02 seconds.
INFO:root:1. Distinct compound target combinations and their counts:
INFO:root:    None: 2 (Compounds: CHEMBL1, CHEMBL9)
INFO:root:    Acetylcholine receptor subunit alpha\Neuronal acetylcholine receptor subunit alpha-3\Neuronal acetylcholine receptor subunit alpha-7\Neuronal acetylcholine receptor; alpha3/beta2\Sigma non-opioid intracellular receptor 1\Voltage-gated inwardly rectifying potassium channel KCNH2: 1 (Compounds: CHEMBL3)
INFO:root:    Adrenergic receptor alpha-1\Sodium channel alpha subunits; brain (Types I, II, III)\Solute carrier family 22 member 1\Voltage-gated inwardly rectifying potassium channel KCNH2: 1 (Compounds: CHEMBL2)
INFO:root:    Aldo-keto reducta

mols[0]={'molecule_chembl_id': 'CHEMBL2', 'molecule_properties': {'alogp': '1.78', 'aromatic_rings': 3, 'full_molformula': 'C19H21N5O4', 'full_mwt': '383.41', 'hba': 8, 'hbd': 1, 'heavy_atoms': 28, 'mw_freebase': '383.41', 'np_likeness_score': '-1.29', 'num_ro5_violations': 0, 'psa': '106.95', 'qed_weighted': '0.73', 'ro3_pass': 'N', 'rtb': 4}, 'molecule_structures': {'canonical_smiles': 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'molfile': '\n     RDKit          2D\n\n 28 31  0  0  0  0  0  0  0  0999 V2000\n    0.9375   -1.9792    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.9292   -2.6917    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.3208   -2.6750    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.2875   -3.0417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.3042   -1.6167    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.3208   -1.9625    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    3.3917   -0.5917    0.0000 C   0  0  0  0  0  0  